In [16]:
import random
import json
from faker import Faker
from datetime import datetime
# Inicializa o gerador de dados fictícios
fake = Faker()

# Definindo intervalo de tempo

start_date = datetime(2024, 8, 1, 0)
end_date = datetime(2024, 8, 1, 23)

def generate_random_data():
    """Gera um dicionário com dados aleatórios no formato JSON especificado."""
    return {
        "imei": str(fake.random_number(digits=15, fix_len=True)),
        "mac": ':'.join([f"{random.randint(0, 255):02X}" for _ in range(6)]),
        "network": random.choice(["VIVO", "TIM", "CLARO", "OI"]),
        "client_ip": fake.ipv4(),
        "latitude": f"{random.uniform(-33.7, 5.3):.6f}",  # Limita a latitude ao Brasil
        "longitude": f"{random.uniform(-73.9, -34.8):.6f}",  # Limita a longitude ao Brasil
        "uid": str(fake.random_number(digits=11, fix_len=True)),
        "password": fake.password(length=6, special_chars=False, digits=True, upper_case=True, lower_case=True),
        "transaction": random.choice(["true", "false"]),
        "api": "login-authentication",
        "endpoint": "v1/login",
        "os": random.choice(["ANDROID", "iOS"]),
        "os_version": str(random.randint(1, 30)),
        "app_version": random.choice(["2.0.1", "2.0.2", "2.0.3"]),
        "error": random.choice(["0", "INCORRECT_PASS", "USER_NOT_FOUND"]),
        "timestamp": str(fake.unix_time(start_datetime=start_date, end_datetime=end_date)*1000)
    }

# Gera um array com 100 dados
data_array = [generate_random_data() for _ in range(10000)]

# Salva o array em um arquivo JSON
with open('data.json', 'w') as f:
    json.dump(data_array, f, indent=4)


In [2]:
import random
import json
from faker import Faker
from datetime import datetime
from azure.eventhub import EventHubProducerClient, EventData
import time
import pandas as pd

# Connection string do Event Hub e o nome do Event Hub
eventhub_name = "fraud-detect"
connection_string = f"[connection-string-here]"

# Caminho para o arquivo CSV
file_path = 'phonedataset.csv'

# Lê o arquivo CSV
df = pd.read_csv(file_path, sep=';')
phone_list = df.values.tolist()

# Inicializa o gerador de dados fictícios
fake = Faker()

# Definindo intervalo de tempo

start_date = datetime(2024, 8, 1, 0)
end_date = datetime(2024, 8, 2, 23)

def generate_random_data():
    """Gera um dicionário com dados aleatórios no formato JSON especificado."""

    brand_version = random.choice(phone_list)
    phone_brand = brand_version[0]
    operational_version = brand_version[1]

    return {
        "imei": str(fake.random_number(digits=15, fix_len=True)),
        "mac": ':'.join([f"{random.randint(0, 255):02X}" for _ in range(6)]),
        "network": random.choice(["VIVO", "TIM", "CLARO", "OI"]),
        "client_ip": fake.ipv4(),
        "latitude": f"{random.uniform(-33.7, 5.3):.6f}",  # Limita a latitude ao Brasil
        "longitude": f"{random.uniform(-73.9, -34.8):.6f}",  # Limita a longitude ao Brasil
        "uid": str(fake.random_number(digits=11, fix_len=True)),
        "password": fake.password(length=6, special_chars=False, digits=True, upper_case=True, lower_case=True),
        "transaction": random.choice(["true", "false"]),
        "api": "login-authentication",
        "endpoint": "v1/login",
        "os": operational_version,
        "phone_brand": phone_brand,
        "app_version": random.choice(["2.0.1", "2.0.2", "2.0.3"]),
        "error": random.choice(["0", "INCORRECT_PASS", "USER_NOT_FOUND"]),
        "timestamp": str(fake.unix_time(start_datetime=start_date, end_datetime=end_date)*1000)
    }


def send_message_to_eventhub(message):
    """Abre client com eventhub e envia batch de mensagens."""
    try:
        producer = EventHubProducerClient.from_connection_string(conn_str=connection_string, eventhub_name=eventhub_name)
    except ValueError as ve:
        print(f"Erro ao criar client: {ve}")
        return 

    try:
        event_data_batch = producer.create_batch()
        event_data_batch.add(EventData(message))
        producer.send_batch(event_data_batch)
    except ValueError as ve:
        print(f"Erro ao enviar mensagem: {ve}")
    except Exception as e:
        print(f"Erro inesperado ao enviar mensagem: {e}")
    finally:
        producer.close()
        print(f"Mensagem enviada: {message}")

for i in range(10000):
    time.sleep(0.5)
    message = json.dumps(generate_random_data())
    print(f"{i} -> {message}")
    send_message_to_eventhub(message)


0 -> {"imei": "979114725589774", "mac": "44:65:C6:BB:17:65", "network": "TIM", "client_ip": "29.223.228.190", "latitude": "-21.384460", "longitude": "-44.329734", "uid": "85669015630", "password": "Tp8EcX", "transaction": "true", "api": "login-authentication", "endpoint": "v1/login", "os": "Android 13", "phone_brand": "Nokia G42", "app_version": "2.0.1", "error": "USER_NOT_FOUND", "timestamp": "1722489320000.0"}
Mensagem enviada: {"imei": "979114725589774", "mac": "44:65:C6:BB:17:65", "network": "TIM", "client_ip": "29.223.228.190", "latitude": "-21.384460", "longitude": "-44.329734", "uid": "85669015630", "password": "Tp8EcX", "transaction": "true", "api": "login-authentication", "endpoint": "v1/login", "os": "Android 13", "phone_brand": "Nokia G42", "app_version": "2.0.1", "error": "USER_NOT_FOUND", "timestamp": "1722489320000.0"}
1 -> {"imei": "131399408334782", "mac": "6B:C2:01:00:57:1C", "network": "VIVO", "client_ip": "25.211.192.89", "latitude": "-11.071146", "longitude": "-73.6

KeyboardInterrupt: 